In [7]:
import sys
import ray
import time
import math
import os


In [8]:
ray.init()

2022-07-26 23:32:35,628	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.28.6.160',
 'raylet_ip_address': '172.28.6.160',
 'redis_address': '172.28.6.160:6379',
 'object_store_address': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707',
 'metrics_export_port': 60383,
 'node_id': '1586b34fa8edaba7ad876283501ca54c33f9f528'}

In [29]:
prefix = ""
@ray.remote
def launch(cc, trace, slow_unit):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation"
    os.chdir(work_dir)
    print("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {}".format(cc, trace, slow_unit))
    os.system("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {}".format(cc, trace, slow_unit))


def gen_trafic_config(wkld, nhost, load, time_s, incast, pattern=0):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/traffic_gen"
    os.chdir(work_dir)
    output_name = "{}_n{}_load{}_t{}_incast{}_pattern{}".format(wkld, nhost, load, time_s, incast, pattern)
    cmd = "python traffic_gen.py -c {}.txt -n {} -l {} -b 100G -t {} -o {}.txt -i {} -p {}".format(wkld, nhost, load, time_s, output_name, incast, pattern)
    os.system(cmd)
    mix_path =  work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation/mix"
    os.system("cp ./{}.txt {}".format(output_name, mix_path))
    return output_name

In [32]:
prefix = gen_trafic_config(wkld="WebSearch_distribution", nhost=64, load=0.3, time_s=0.100, incast=0, pattern=2)
for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
    trace = prefix
    if (method != "slow"):   
        f_id = launch.remote(method, trace, 0)
    else:
        f_id = launch.remote("abc", trace, 1)

without incast: 14024
with incast: 14024
(pid=38421) python run.py --c dctcp --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 0
(pid=38408) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 1
(pid=38377) python run.py --c dcqcn --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 0
(pid=38373) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 0
(pid=38415) python run.py --c hp --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 0
(pid=38404) python run.py --c timely --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2  --bw 100 --topo 2layer --hpai 80 --slow_unit 0


(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38408) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38373) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38415) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) 'build' finished successfully (0.384s)
(pid=38408) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38408) 'build' finished successfully (0.384s)
(pid=38373) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-

(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2_dctcp.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2_dctcp.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern2_dctcp.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		8
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.0625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				10Mb/s
(pid=38421) RATE_HAI			10Mb/s
(pid=38421) MIN_RATE		1000Mb/s
(pid=38421) DCTCP_RATE_AI				61

(pid=38421) maxRtt=8320 maxBdp=104000
(pid=38421) Running Simulation.
(pid=38408) maxRtt=8320 maxBdp=104000
(pid=38408) Running Simulation.
(pid=38377) maxRtt=8320 maxBdp=104000
(pid=38377) Running Simulation.
(pid=38373) maxRtt=8320 maxBdp=104000
(pid=38373) Running Simulation.
(pid=38415) maxRtt=8320 maxBdp=104000
(pid=38415) Running Simulation.
(pid=38404) maxRtt=8320 maxBdp=104000
(pid=38404) Running Simulation.


In [18]:
f_id = launch.remote("abc", trace, 1)

(pid=38404) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) 'build' finished successfully (0.361s)


(pid=38404) ENABLE_QCN			Yes
(pid=38404) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38404) PACKET_PAYLOAD_SIZE		1000
(pid=38404) TOPOLOGY_FILE			mix/2layer.txt
(pid=38404) FLOW_FILE			mix/WebSearch_distribution_n64_load0.3_t0.1_incast0_onlycrosspod1.txt
(pid=38404) TRACE_FILE			mix/trace.txt
(pid=38404) TRACE_OUTPUT_FILE		mix/mix_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_onlycrosspod1_abc_slowUnit.tr
(pid=38404) FCT_OUTPUT_FILE		mix/fct_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_onlycrosspod1_abc_slowUnit.txt
(pid=38404) PFC_OUTPUT_FILE				mix/pfc_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_onlycrosspod1_abc_slowUnit.txt
(pid=38404) SIMULATOR_STOP_TIME		2.1
(pid=38404) CC_MODE		9
(pid=38404) ALPHA_RESUME_INTERVAL		1
(pid=38404) RATE_DECREASE_INTERVAL		4
(pid=38404) CLAMP_TARGET_RATE		No
(pid=38404) RP_TIMER			300
(pid=38404) EWMA_GAIN			0.00390625
(pid=38404) FAST_RECOVERY_TIMES		1
(pid=38404) RATE_AI				20Mb/s
(pid=38404) RATE_HAI			200Mb/s
(pid=38404) MIN_